In [ ]:
import cv2
import numpy as np

import pytesseract
# Configuration des paramètres du modèle YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Chargement des classes
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Configuration des paramètres du modèle
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Configuration de la capture vidéo à partir de la caméra
cap = cv2.VideoCapture(0)

# Configuration de Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

while True:
    # Capture de la frame depuis la caméra
    ret, frame = cap.read()

    # Prétraitement de l'image
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Passage de l'image prétraitée dans le réseau de neurones
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Détection des objets et filtrage des plaques d'immatriculation
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "license plate":
                # Calcul des coordonnées du rectangle englobant
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Ajout des informations dans les listes
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Application de la non-maxima suppression pour éliminer les détections redondantes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Extraction du texte de la plaque d'immatriculation
    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]

        # Extraction de la région de la plaque d'immatriculation
        plate_img = frame[y:y+h, x:x+w]

        # Conversion de l'image en niveaux de gris
        gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)

        # Amélioration du contraste de l'image
        gray = cv2.equalizeHist(gray)

        # Extraction du texte en utilisant Tesseract OCR
        license_plate = pytesseract.image_to_string(gray, config='--psm 7')

        # Affichage de la plaque d'immatriculation et du texte extrait
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, license_plate, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Affichage de la frame avec les résultats de la détection
    cv2.imshow("License Plate Detection", frame)

    # Sortir de la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()


: 